In [1]:
# ! pip install python-dotenv
# ! pip install -qU langchain-google-vertexai
! pip install langchain_groq
! pip install langchain
! pip install langchain_community



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import FakeEmbeddings

In [3]:
# Load the environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
model_version = "llama-3.3-70b-versatile"


In [4]:
# Create Model Instance and Invoke a message to the model
model = ChatGroq(api_key=GROQ_API_KEY, model=model_version)
# model.invoke("Hello, how are you?")

# Parse the output through the STROutputParser
parser = StrOutputParser()
chain = model|parser
chain.invoke("Hello, how are you?")

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


"Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm here and ready to help with any questions or topics you'd like to discuss. How can I assist you today?"

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: 

In [5]:
# Testing the Models ability to recollect
# chain.invoke("Hello, I'm Emmanuel")

from langchain_core.messages import AIMessage, HumanMessage
chain.invoke(
    [
        HumanMessage("Hello,I'm Emmanuel"),
        AIMessage("Hello Emmanuel, it's nice to meet you. Is there something I can help you with or would you like to chat?"),
        HumanMessage("What is my Name?")
    ]
)

'Your name is Emmanuel.'

In [ ]:
# Using Langgraph Message Persistence.
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define A Graph
workflow = StateGraph(state_schema=MessagesState)

# Define the Function That calls the model
def call_quaver(state: MessagesState):
    response = chain.invoke(state["messages"])
    return {"messages": response}

#define the single node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_quaver)

# add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}


In [8]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================ Human Message =================================

Hello Bob. It's nice to meet you. Is there something I can help you with or would you like to chat?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Your name is Bob. You told me that when you first said hello.


In [11]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

You're Bob. We established that earlier in our conversation.


In [16]:
## Adding a Prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
        "You're a Chess Grandmaster with a great sense of humor.",
        ),
        MessagesPlaceholder(variable_name="messages"),

    ]
)

In [17]:
# Update our Application to incorporate the prompt template
workflow = StateGraph(state_schema=MessagesState) 

def call_quaver(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = chain.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_quaver)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim. Write a short commentary like Gothan Chess using the first 5 moves of the italian game."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Jim, my friend, it's a pleasure to meet you. I hope you're ready to get checkmated... I mean, acquainted. Don't worry, I won't make too many chess puns. Unless, of course, you want to engage in a battle of wits and see who comes out on top. In that case, let's set up the board and may the best player win! What brings you to the table, Jim?
